<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo


1.   Crear sus propios vectores con Gensim basado en lo visto en clase con otro
2.   Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
3.   Graficarlos





dataset.
(sacar conclusiones entre palabras similitudes y diferencias).
Graficarlos.
Obtener conclusiones.

In [ ]:
pip install gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 26.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

In [ ]:
artistas=os.listdir("./songs_dataset/")

In [ ]:
artistas[0]

'beatles.txt'

In [ ]:
from pathlib import Path
import pandas as pd

base = Path("songs_dataset")
dic_art_doc = {}

for fname in artistas:
    path = base / fname
    # nombre "bonito" del artista (opcional)
    artist = path.stem.replace('-', ' ').replace('_', ' ').title()

    # leer líneas del txt → DataFrame
    try:
        lines = path.read_text(encoding="utf-8").splitlines()
    except UnicodeDecodeError:
        lines = path.read_text(encoding="latin-1").splitlines()

    df = pd.DataFrame({"song": lines})
    dic_art_doc[artist] = df  # o dic_art_doc[fname] si preferís la clave = archivo



# Creamos un dataframe para visualizar por cada artista la cantidad de documentos existentes

In [ ]:
df_artista_documentos = pd.DataFrame({
    "artist": list(dic_art_doc.keys()),
    "n_docs": [df.shape[0] for df in dic_art_doc.values()]
}).sort_values("artist").reset_index(drop=True)

In [ ]:
 df_artista_documentos

,artist,n_docs
0,Adele,2400
1,Al Green,2130
2,Alicia Keys,2897
3,Amy Winehouse,1910
4,Beatles,1846
5,Bieber,3715
6,Bjork,1693
7,Blink 182,2332
8,Bob Dylan,6038
9,Bob Marley,2218


# Selecionamos para analizar el courpus de el Artista Prince

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/prince.txt', sep='/n', header=None)
#df.to_excel('songs_dataset/prince.xlsx')

/tmp/ipython-input-41537928.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [ ]:
songs_dataset/prince.txt

NameError: name 'songs_dataset' is not defined

In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9616


# 1.   Crear sus propios vectores con Gensim basado en lo visto en clase con otro

### Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:32]

### Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
#
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9616


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1204


# 2 Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

 Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 256272.53125
Loss after epoch 1: 176920.71875
Loss after epoch 2: 175195.3125
Loss after epoch 3: 174172.0
Loss after epoch 4: 170253.0
Loss after epoch 5: 157612.8125
Loss after epoch 6: 142256.0
Loss after epoch 7: 138182.75
Loss after epoch 8: 134005.25
Loss after epoch 9: 130863.875
Loss after epoch 10: 128808.25
Loss after epoch 11: 125939.5
Loss after epoch 12: 123857.5
Loss after epoch 13: 116720.0
Loss after epoch 14: 110051.0
Loss after epoch 15: 109010.75
Loss after epoch 16: 107527.25
Loss after epoch 17: 107130.0
Loss after epoch 18: 106924.25
Loss after epoch 19: 106332.0


(819439, 1263020)

Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('without', 0.6551504731178284),
 ('critics', 0.631041407585144),
 ('forever', 0.6221495866775513),
 ('color', 0.6142560839653015),
 ('calling', 0.5995683670043945),
 ('lie', 0.5955805778503418),
 ('loving', 0.5880191326141357),
 ('because', 0.58735191822052),
 ('loved', 0.5870353579521179),
 ('machine', 0.5868354439735413)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[("y'all", 0.1173865869641304),
 ('mama', 0.11113794893026352),
 ('tres', 0.0977240577340126),
 ('belle', 0.09174241870641708),
 ('zero', 0.07758185267448425),
 ('thunder', 0.0763014405965805),
 ('everybody', 0.07395172864198685),
 ('etes', 0.06593500822782516),
 ('girls', 0.05560768395662308),
 ('housequake', 0.05386542156338692)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["baby"], topn=10)

[('sweet', 0.7661840915679932),
 ('help', 0.68369060754776),
 ('teach', 0.6793935894966125),
 ('sorry', 0.6639257669448853),
 ('cha', 0.6634940505027771),
 ("fallin'", 0.6618184447288513),
 ('crazy', 0.6506999731063843),
 ('drive', 0.647493302822113),
 ("'cuz", 0.6471139788627625),
 ('yea', 0.6383155584335327)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["baby"], topn=5)

[("i've", 0.09030059725046158),
 ('was', 0.06368812173604965),
 ('as', 0.053862251341342926),
 ('been', 0.051157396286726),
 ('seen', 0.045232150703668594)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('without', 0.6551504731178284),
 ('critics', 0.6310414671897888),
 ('forever', 0.6221495866775513),
 ('color', 0.6142560839653015),
 ('calling', 0.5995683670043945),
 ('lie', 0.5955805778503418),
 ('loving', 0.5880191326141357),
 ('because', 0.58735191822052),
 ('loved', 0.5870353579521179)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('without', 0.6551504731178284),
 ('critics', 0.631041407585144),
 ('forever', 0.6221495866775513),
 ('color', 0.6142560839653015),
 ('calling', 0.5995683670043945),
 ('lie', 0.5955805778503418),
 ('loving', 0.5880191326141357),
 ('because', 0.58735191822052),
 ('loved', 0.5870353579521179),
 ('machine', 0.5868354439735413)]

# Conclusiones Love .
En un modelo Word2Vec, dos palabras son similares si aparecen en contextos parecidos, es decir, rodeadas de palabras similares dentro de una ventana de texto.
Por eso, la similitud no mide sinónimos exactos, sino co-ocurrencias semánticas o temáticas.

La similitud de “love” con estas palabras surge no por significado léxico directo, sino por co-ocurrencia sistemática en contextos similares dentro de las letras.
Word2Vec identifica patrones sintácticos y temáticos repetidos (por ejemplo, “without love”, “love forever”, “because I love you”) y aproxima sus vectores en el espacio semántico.
En consecuencia, los resultados reflejan la estructura lingüística del corpus —centrada en expresiones románticas, relacionales y sensoriales— más que una conexión conceptual universal del término “love”.

# Conclusiones Baby .
La similitud entre “baby” y estas palabras se debe a su frecuente co-ocurrencia en estructuras afectivas, emocionales y rítmicas dentro del corpus.
El modelo Word2Vec no está capturando “significados” propiamente dichos, sino patrones de uso en el lenguaje lírico.
En este caso, baby actúa como un vocativo recurrente que aparece en contextos de:

Expresiones de cariño o deseo (sweet, fallin’, crazy)

Pedidos o disculpas (help, sorry, teach)

Marcadores fonéticos o rítmicos (’cuz, yea, cha)

En otras palabras:
 baby se encuentra en el centro de un campo semántico de intimidad emocional y musicalidad.
El modelo identifica que alrededor de “baby” giran palabras que construyen la interacción afectiva directa típica del estilo de Prince (diálogo íntimo, deseo, arrepentimiento, ritmo).




### 3 Graficarlos.

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=100
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

# Conclusiones generales del análisis Word2Vec
1. Estructura semántica del corpus

El modelo Word2Vec entrenado sobre el corpus de letras musicales (probablemente de artistas como Prince) revela un espacio semántico coherente y altamente temático.
Los vectores de palabras se agrupan en torno a núcleos conceptuales recurrentes, como:

Emociones y relaciones afectivas: love, baby, sorry, crazy, forever, sweet.

Acciones humanas o verbos de interacción: drive, teach, help, call, fallin.

Lenguaje coloquial y musical: yeah, cuz, cha, yea, uh.

Estos grupos surgen naturalmente del patrón de co-ocurrencias, lo cual indica que el modelo captó la estructura lingüística y emocional del género musical, más que relaciones léxicas tradicionales.

2. Distribución y visualización (gráfico de los 200 vectores)

En la proyección 2D (PCA o t-SNE) de los 200 vectores más frecuentes, se observan clústeres claramente diferenciados, que corresponden a distintos campos semánticos o emocionales:

Región aproximada	Tipo de palabras dominantes	Interpretación
 Afecto y deseo	love, baby, sweet, crazy, forever	Eje principal del corpus: el lenguaje emocional y romántico.
Conflicto y disculpa	sorry, lie, without, pain	Expresiones de pérdida, culpa o ruptura sentimental.
 Ritmo y expresividad	yeah, cha, uh, ’cuz	Elementos fónicos típicos de la música pop/funk que marcan compás o estilo.
 Metáfora y abstracción	color, machine, dream	Palabras usadas en contextos poéticos o simbólicos, frecuentes en la lírica de Prince.

La separación de estos grupos confirma que el modelo aprendió representaciones distribuidas significativas, donde la proximidad refleja afinidad temática y emocional.

3. Funcionamiento semántico del modelo

El análisis de las similitudes mostró que:

Word2Vec no mide sinonimia, sino contexto compartido.

Palabras con alta similitud con love o baby comparten entornos lingüísticos similares, como frases del tipo “sorry baby”, “without love”, “crazy for you”.

Esto demuestra que el modelo aprende regularidades gramaticales y estilísticas, además de relaciones semánticas.

En otras palabras:

El espacio vectorial refleja cómo se usa el lenguaje dentro del corpus, no cómo se define en un diccionario.

4. Valor interpretativo del modelo

El resultado evidencia que el corpus tiene una coherencia temática fuerte (amor, deseo, emoción, ritmo), y que el modelo Word2Vec logró capturarla con eficacia.
El análisis gráfico y de similitudes sugiere que:

Las letras comparten estructuras repetitivas y fonéticas, que el modelo traduce en agrupamientos consistentes.

Los términos centrales (love, baby) actúan como anclas semánticas, alrededor de las cuales orbitan palabras afectivas, verbos de acción y partículas rítmicas.

El espacio vectorial obtenido es una representación empírica del estilo lingüístico-musical del corpus